### Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** до 30 апреля 2018, 06:00   
**Штраф за опоздание:** -2 балла после 06:00 30 апреля, -4 балла после 06:00 7 мая, -6 баллов после 06:00 14 мая, -8 баллов после 06:00 21 мая

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush


Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (3 балла)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на wine и Speed Dating Data.

###### Задание 2 (3 балла)
Добиться скорости работы на fit не медленнее чем в 10 раз sklearn на данных wine и Speed Dating Data. 
Для этого используем numpy.

###### Задание 3 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 4 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 4 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw3.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [11]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
%load_ext pycodestyle_magic

The pycodestyle_magic extension is already loaded. To reload it, use:
  %reload_ext pycodestyle_magic


In [36]:
#%%pycodestyle


class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None, sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return  1 - np.sum((l_c ** 2) / (l_s * (l_s + r_s)) + (r_c ** 2) / (r_s * (l_s + r_s)), axis=1)
    
    def __entropy(self, l_c, l_s, r_c, r_s):
        return -np.sum(l_c/(l_s + r_s)*np.log2(l_c/l_s), axis=1)-np.sum(r_c/(l_s + r_s)*np.log2(r_c/r_s), axis=1)

    def __misclass(self, l_c, l_s, r_c, r_s):
        return 1 - np.max(l_c / (l_s + r_s), axis=1) - np.max(r_c / (l_s + r_s), axis=1)

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = np.arange(n_feature)
        np.random.shuffle(feature_ids)
        return np.array(feature_ids[:int(np.sqrt(n_feature))])
        
    def __get_feature_ids_log2(self, n_feature):
        feature_ids = np.arange(n_feature)
        np.random.shuffle(feature_ids)
        return np.array(feature_ids[:int(np.log2(n_feature))])

    def __get_feature_ids_N(self, n_feature):
        return np.arange(n_feature)
    
    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        x_sort, y_sort = self.__sort_samples(x,y)
        pos_splits = np.where(y_sort[:-1] != y_sort[1:])[0] + 1
        if pos_splits.size == 0:
            return np.inf, None
        classes = np.zeros((pos_splits.shape[0],self.num_class))
        classes[np.arange(pos_splits.shape[0]), y_sort[pos_splits - 1]] = 1
        elements = pos_splits - np.append([0], pos_splits[:-1])
        classes = classes * elements.reshape(-1,1)
        l_class_count = np.cumsum(classes, axis=0)
        r_class_count = np.bincount(y_sort, minlength=self.num_class) - l_class_count
        l_class_size = pos_splits.reshape((pos_splits.shape[0],1))
        r_class_size = y_sort.shape[0] - l_class_size
        gs = self.G_function(l_class_count, l_class_size, r_class_count, r_class_size)
        indx = np.argmin(gs)
        x_indx = pos_splits[indx]
        return gs[indx], (x_sort[x_indx - 1] + x_sort[x_indx]) / 2.0      
        
    def __fit_node(self, x, y, node_id, depth):
        if (self.max_depth is not None and depth == self.max_depth or \
            y.size < self.min_samples_split):
            self.tree[node_id] = (self.LEAF_TYPE, np.bincount(y).argmax(), np.bincount(y).astype(float)/y.size)
            return
        pos_thresholds = np.array([self.__find_threshold(x[:, i], y) for i in self.get_feature_ids(x.shape[1])])
        best_threshold_id = pos_thresholds[:,0].argmin()
        best_threshold = pos_thresholds[best_threshold_id, 1]
        if best_threshold == None:
            self.tree[node_id] = (self.LEAF_TYPE, np.bincount(y).argmax(), np.bincount(y).astype(float) / y.size)
        else:
            x_l, x_r, y_l, y_r = self.__div_samples(x, y, best_threshold_id, best_threshold)
            if (x_l.size == 0 or x_r.size == 0):
                self.tree[node_id] = (self.LEAF_TYPE, np.bincount(y).argmax(), np.bincount(y).astype(float)/y.size)
                return
            else:
                self.tree[node_id] = (self.NON_LEAF_TYPE,best_threshold_id, best_threshold)
                self.__fit_node(x_l, y_l,2 * node_id + 1, depth + 1)
                self.__fit_node(x_r, y_r, 2 * node_id + 2, depth + 1)
            if self.G_function == self.__gini:
                g=(1 - np.sum(np.unique(y, return_counts=True)[1]** 2 / y.size ** 2))
                g_l=(1 - np.sum(np.unique(y_l, return_counts=True)[1] ** 2 / y_l.size ** 2))
                g_r=(1 - np.sum(np.unique(y_r, return_counts=True)[1] ** 2 / y_r.size ** 2))
                self.feature_importances_[best_threshold_id] += (y.size*g - y_l.size*g_l - y_r.size*g_r)
            if self.G_function == self.__entropy:
                g=-(np.sum((np.unique(y, return_counts=True)[1] / y.size) * np.log2(np.unique(y, return_counts=True)[1] / y.size)))
                g_l=-(np.sum((np.unique(y_l, return_counts=True)[1] / y_l.size) * np.log2(np.unique(y_l, return_counts=True)[1] / y_l.size)))
                g_r=-(np.sum((np.unique(y_r, return_counts=True)[1] / y_r.size) * np.log2(np.unique(y_r, return_counts=True)[1] / y_r.size)))
                self.feature_importances_[best_threshold_id] += (y.size*g - y_l.size*g_l - y_r.size*g_r)   
            if self.G_function == self.__misclass:
                g=(1 - np.max(np.sum(np.unique(y, return_counts=True)[1] / y.size)))
                g_l=(1 - np.max(np.sum(np.unique(y_l, return_counts=True)[1] / y_l.size)))
                g_r=(1 - np.max(np.sum(np.unique(y_r, return_counts=True)[1] / y_r.size)))
                self.feature_importances_[best_threshold_id] += (y.size*g - y_l.size*g_l - y_r.size*g_r)
                 
    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.feature_importances_ = np.zeros(x.shape[1])
        self.__fit_node(x, y, 0, 0) 
        self.feature_importances_ /= y.size

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]
        
    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])
    
    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [37]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [38]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

## Проверка скорости работы на wine

In [39]:
%time clf.fit(X_train, y_train)

CPU times: user 4.74 ms, sys: 48 µs, total: 4.79 ms
Wall time: 3.87 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [40]:
%time my_clf.fit(X_train, y_train)

CPU times: user 13.5 ms, sys: 94 µs, total: 13.6 ms
Wall time: 12.9 ms


## Проверка качества работы на wine

In [41]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.8222222222222221

In [42]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.8857142857142858

## Подготовка данных Speed Dating Data 

In [129]:
df = pd.read_csv('speed-dating-experiment/Speed Dating Data.csv', encoding='cp1251')
for i in df.columns:
    if (type(df[i][0]) == np.float64 or type(df[i][0]) == np.int64 or type(df[i][0]) == float):
        continue
    else:
        df.drop(i, axis = 1, inplace = True)
for i in df.columns:
    if df[i].isnull().sum()>500:
        df.drop(i,axis=1, inplace=True)

In [130]:
df.tail()

,iid,id,gender,idg,condtn,wave,round,position,order,partner,...,intel3_1,amb3_1,dec,attr,sinc,intel,fun,like,prob,met
8373,552,22.0,1,44,2,21,22,14,5,18,...,7.0,7.0,0,3.0,5.0,5.0,5.0,2.0,5.0,0.0
8374,552,22.0,1,44,2,21,22,13,4,19,...,7.0,7.0,0,4.0,6.0,8.0,4.0,4.0,4.0,0.0
8375,552,22.0,1,44,2,21,22,19,10,20,...,7.0,7.0,0,4.0,7.0,8.0,8.0,6.0,5.0,0.0
8376,552,22.0,1,44,2,21,22,3,16,21,...,7.0,7.0,0,4.0,6.0,5.0,4.0,5.0,5.0,0.0
8377,552,NaN,1,44,2,21,22,2,15,22,...,7.0,7.0,0,3.0,7.0,6.0,4.0,4.0,5.0,0.0


In [131]:
df=df.fillna(-100000)

In [132]:
y = df.loc[:, 'match'].values
df.drop(['match','dec','dec_o','like_o', 'like'], axis=1, inplace = True)

In [133]:
X=df.values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

## Проверка скорости работы на Speed Dating Data 

In [134]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [135]:
%time clf.fit(X_train, y_train)

CPU times: user 157 ms, sys: 3.87 ms, total: 161 ms
Wall time: 160 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [136]:
%time my_clf.fit(X_train, y_train)

CPU times: user 1.32 s, sys: 3 µs, total: 1.32 s
Wall time: 1.32 s


## Проверка качества работы на Speed Dating Data

In [137]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.6412754375593904

In [138]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.6516960506217885

## Задание 4


In [139]:
print(pd.Series(data=my_clf.feature_importances_, 
        index=df.columns[:]).sort_values(ascending=False)[:10])
print(pd.Series(data=clf.feature_importances_,
          index=df.columns[:]).sort_values(ascending=False)[:10])

fun_o       0.016263
attr_o      0.015847
fun         0.015127
attr        0.013096
prob_o      0.008620
iid         0.004420
prob        0.003041
id          0.002923
fun1_1      0.002517
position    0.002115
dtype: float64
attr_o      0.082975
fun         0.077647
attr        0.051465
prob        0.048063
prob_o      0.042853
fun_o       0.038687
partner     0.029444
order       0.028946
int_corr    0.028372
pf_o_sha    0.023112
dtype: float64


## Задание 5

In [140]:
parametrs = {
    "min_samples_split": [i for i in range(2, 50, 2)],
    "max_depth": [i for i in range(2, 20)],
    "n_estimators": [i for i in range(100, 150,10)],
    "criterion": ["gini", "entropy"]
}
RS = RandomizedSearchCV(RandomForestClassifier(random_state=123), param_distributions=parametrs, n_iter=70)
RS.fit(X_train, y_train)
print(RS.best_params_)

/home/serega/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


{'n_estimators': 100, 'min_samples_split': 4, 'max_depth': 17, 'criterion': 'gini'}
